<a href="https://colab.research.google.com/github/nashwasalih/codealpha/blob/main/securecodingreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
import hashlib
import os

# ============================================================
# PART 1 – Vulnerable Application (Code to Audit)
# ============================================================

vulnerable_code = """
import sqlite3

def login(username, password):
    conn = sqlite3.connect("users.db")
    cursor = conn.cursor()

    # SQL Injection vulnerability
    query = f"SELECT * FROM users WHERE username='{username}' AND password='{password}'"
    cursor.execute(query)

    user = cursor.fetchone()
    conn.close()

    if user:
        return "Login Success"
    else:
        return "Login Failed"

# Hardcoded secret key
SECRET_KEY = "mysecret123"

# Plaintext password example
admin_password = "admin123"
"""

# Save vulnerable code to file
with open("vulnerable_app.py", "w") as f:
    f.write(vulnerable_code)

print("Vulnerable application created: vulnerable_app.py")


# ============================================================
# PART 2 – Static Code Analyzer (Manual Inspection Tool)
# ============================================================

def analyze_code(filename):
    print("\n===== SECURE CODING REVIEW REPORT =====\n")

    with open(filename, "r") as file:
        code = file.read()

    issues_found = []

    # 1️⃣ Detect SQL Injection (f-string queries)
    if re.search(r"f\"SELECT .*{.*}\"", code):
        issues_found.append(("SQL Injection", "High",
                             "Dynamic SQL query using string formatting."))

    # 2️⃣ Detect Hardcoded Secrets
    if re.search(r"SECRET_KEY\s*=\s*\".*\"", code):
        issues_found.append(("Hardcoded Secret Key", "High",
                             "Secret key is hardcoded in source code."))

    # 3️⃣ Detect Plaintext Passwords
    if re.search(r"password\s*=\s*\".*\"", code):
        issues_found.append(("Plaintext Password Storage", "High",
                             "Password stored without hashing."))

    # 4️⃣ Missing Password Hashing
    if "hashlib" not in code:
        issues_found.append(("No Password Hashing", "Medium",
                             "Passwords are not hashed before storage."))

    # 5️⃣ No Input Validation
    if "input(" not in code:
        issues_found.append(("No Input Validation", "Medium",
                             "No validation for user inputs."))

    # Display Findings
    if not issues_found:
        print("No major vulnerabilities found.")
    else:
        for idx, issue in enumerate(issues_found, 1):
            print(f"{idx}. Issue: {issue[0]}")
            print(f"   Risk Level: {issue[1]}")
            print(f"   Description: {issue[2]}\n")

    print("===== RECOMMENDATIONS =====\n")

    print("1. Use Parameterized Queries:")
    print("   cursor.execute('SELECT * FROM users WHERE username=? AND password=?', (username, password))\n")

    print("2. Hash Passwords Using hashlib or bcrypt:")
    print("   hashed_password = hashlib.sha256(password.encode()).hexdigest()\n")

    print("3. Store Secrets in Environment Variables:")
    print("   SECRET_KEY = os.getenv('SECRET_KEY')\n")

    print("4. Validate and Sanitize All User Inputs\n")

    print("5. Use Static Analysis Tools:")
    print("   - Bandit (bandit vulnerable_app.py)")
    print("   - SonarQube")
    print("   - pylint\n")

    print("===== REMEDIATION STEPS =====")
    print("• Replace dynamic SQL with prepared statements.")
    print("• Hash and salt passwords.")
    print("• Remove hardcoded credentials.")
    print("• Implement proper input validation.")
    print("• Follow OWASP Secure Coding Guidelines.\n")

    print("===== REVIEW COMPLETED =====")


# Run Secure Coding Review
analyze_code("vulnerable_app.py")



Vulnerable application created: vulnerable_app.py

===== SECURE CODING REVIEW REPORT =====

1. Issue: Hardcoded Secret Key
   Risk Level: High
   Description: Secret key is hardcoded in source code.

2. Issue: Plaintext Password Storage
   Risk Level: High
   Description: Password stored without hashing.

3. Issue: No Password Hashing
   Risk Level: Medium
   Description: Passwords are not hashed before storage.

4. Issue: No Input Validation
   Risk Level: Medium
   Description: No validation for user inputs.

===== RECOMMENDATIONS =====

1. Use Parameterized Queries:
   cursor.execute('SELECT * FROM users WHERE username=? AND password=?', (username, password))

2. Hash Passwords Using hashlib or bcrypt:
   hashed_password = hashlib.sha256(password.encode()).hexdigest()

3. Store Secrets in Environment Variables:
   SECRET_KEY = os.getenv('SECRET_KEY')

4. Validate and Sanitize All User Inputs

5. Use Static Analysis Tools:
   - Bandit (bandit vulnerable_app.py)
   - SonarQube
   - py